In [1]:
# Imports
from os import link
import pandas as pd
import numpy as np
import datetime

In [10]:
import pandas as pd
df_test = pd.read_csv('/Users/finnzurmuehlen/Downloads/temp_tweet_database_08_31.csv', lineterminator='\n', low_memory=False)
df_test

,Unnamed: 0,Unnamed: 0.1,source,lang,tweet_created_at,author_id,text,tweet_id,public_metrics.retweet_count,public_metrics.reply_count,...,location,party,withheld.copyright,withheld.country_codes_x,withheld.scope_x,withheld.country_codes_y,withheld.scope_y,withheld.country_codes,withheld.scope,sentiment
0,0,0,Twitter for Android,de,2021-08-30T23:59:03.000Z,1082371703181844480,SPD 40% Wann? https://t.co/ShYjuURkmS,1432493097821245445,0,2,...,NaN,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
1,1,1,Eauto Check,de,2021-08-30T23:58:56.000Z,1552359542,https://t.co/208fqUCvpj Scholz im Triell wie M...,1432493065994973188,0,0,...,Lutherstadt Wittenberg,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
2,2,2,Twitter for Android,de,2021-08-30T23:58:34.000Z,848002447385735168,"@BBacterioO Weil das pfui ist, so wie links se...",1432492973674147843,0,0,...,sachsn || he/er,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
3,3,3,Twitter for Android,de,2021-08-30T23:58:08.000Z,2912522008,@darioschramm Dabei dann bitte Mal anschauen w...,1432492867444948993,0,0,...,Bremen,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
4,4,4,Twitter for iPad,de,2021-08-30T23:57:48.000Z,546683248,@dervolkan @JoergWimalasena @larsweisbrod Ziem...,1432492782581596168,5,1,...,Berlin,SPD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61563,61563,61563,Twitter Web App,de,2021-08-30T00:24:36.000Z,714433337356599296,btw: Die #KAMPA2021 war sehr gut!\n@DiePARTEI_...,1432137140876546062,0,1,...,Bremen,OTHERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
61564,61564,61564,Twitter Web App,de,2021-08-30T00:24:33.000Z,714433337356599296,^^Irgendwann zwischen meiner 2. &amp; 3. (erfo...,1432137125038891008,1,1,...,Bremen,OTHERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
61565,61565,61565,Twitter Web App,de,2021-08-30T00:24:30.000Z,714433337356599296,#Wahlkriegsmeldung: \nNeulich bei #TeamOlaf (#...,1432137116142776320,0,1,...,Bremen,OTHERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,negative
61566,61566,61566,Twitter for iPad,de,2021-08-30T00:08:49.000Z,712637855672242176,@holgertma Er konnte schon seit längerer Zeit ...,1432133169390788616,0,0,...,"Dresden, Sachsen",OTHERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral
